In [95]:
# Imports
import requests
import pandas as pd

In [96]:
# Global Variables
api_key = "RGAPI-cf22cdfb-ba06-44f4-b568-96d5293fb308"
base_url = "https://americas.api.riotgames.com"
tag_line = "NA1"

In [133]:
# Function to get unique PUUID by summoner name
def get_puuid_by_name(gameName, tag_line):
    url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tag_line}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    puuid = response.json()["puuid"]
    return puuid


# Function to get summoner data by PUUID
def get_summoner_by_puuid(puuid):
    url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    return response.json()


# Function to get rank data by summoner ID
def get_rank_by_summoner_id(puuid):
    url = f"https://na1.api.riotgames.com/lol/league/v4/entries/by-puuid/{puuid}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    return response.json()


# Function to get match history by PUUID
def get_matches_by_puuid(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    matches = response.json()
    return matches


# Function to get match data by match ID
def get_match_data_by_match_id(match_id):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    match_data = response.json()
    return match_data

In [146]:
# Function to get the PUUIDs of the participants in a match
def get_match_puuids(match_id):
    match_data = get_match_data_by_match_id(match_id)
    return match_data["metadata"]["participants"]

In [147]:
test_puuid = get_puuid_by_name("ARMAO", tag_line)
matches = get_matches_by_puuid(test_puuid)
match_data = get_match_data_by_match_id(matches[0])

print(get_match_puuids(matches[0]))



['kmsFAO_77jjUMeGKQ-ajFOx-EXjF7bYBXZ5MYktRPLy1DFqqq5Y7dO3SgHtNVXzQ4Wgc1J2NVgjLZA', '-7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ', 'KZY9vPlc8JNM4HRcsYEpMfn-0RB48gg7sMP_o7ezc0vnIjLfPwrW3JuqB8eIOdxofJgzlgxLCcaJWw', 'sDeS5zC_lb_nIy9QbnpEihKAQZsdCaLQeZ_RP7vY-mBCOFlPZsV3BMSATDty6dZUrY1OmmFMPhLVIw', 'ntNWuQm-xSDMZqTZKKmciU6DAcGZD7_r1D6nhCTHpEiLyhqCssmO5gAvkHtgWuMK4pqt3Yx-E_F34w', 'XGObXx2C5ckGP_Avqvw29YmV_pyj8uR5VaGvDzRUec8Up0TAGrpGgsCwxEsoGC5kwAePOuDmgroZjw', '5qfT43Vod8IuFlD4X8KWZlANpo9IhVl00LOSxco_526ap3a9qqDN56ozIhWa4ZUGuAUWFUR9qlimWw', 'TawUDMlM0jPs111Y_mcZoB7y-5c0Xw-SdVc-xOgBPRD36Uqd7O0rz73OR4ke46Gh7vQPGz5zUPZXLA', 'NmnZXDZMkHV9vjy54eXr6dxIS12VAF8CTbSej4ZR2d0FGa-EApZTESKGAAEz2bjbooQfZnfmsxlO8g', 'KpB95ghFhG_p6IG2eP138IZC354vpvH-ys_giEnoMlTQn0SN5iWgMY9bqvwYDaIl0RTMrPN3za5ZtA']
